<a href="https://colab.research.google.com/github/gokultp/malayalam_songs_gpt/blob/master/Malayalam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/gokultp/malayalam_songs_gpt/master/data/songs.txt

--2024-06-10 08:11:10--  https://raw.githubusercontent.com/gokultp/malayalam_songs_gpt/master/data/songs.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34085911 (33M) [text/plain]
Saving to: ‘songs.txt’

songs.txt           100%[===================>]  32.51M   194MB/s    in 0.2s    

2024-06-10 08:11:11 (194 MB/s) - ‘songs.txt’ saved [34085911/34085911]



Install and setup sentence piece tokeniser

In [2]:
with open('songs.txt') as f:
  text = f.read()

In [10]:
text[101:148]

' ഹരിഗോവിന്ദാ..ഗോവിന്ദാ ഹരിഗോവിന്ദാ..(4)\nസമാനഹൃദ'

In [3]:
!pip install torch tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [11]:
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")


In [28]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 48 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

Adding a little optimiation here. We are training with only malayalam characters. The tiktoken has nearly 100k vocabs and all are not usefull for us

Our vocab is much smaller. So writing a custom encoder to reduce the complexity

In [13]:
uniq_vocabs = {num: token for num, token in enumerate(list(set(encoder.encode(text)))) }

tok_to_id = {token: num for num, token in uniq_vocabs.items()}


In [ ]:
uniq_vocabs

In [14]:
encode = lambda txt : [tok_to_id[y] for y in encoder.encode(txt)]
decode = lambda id_lst: encoder.decode([uniq_vocabs[y] for y in id_lst])



Test if both encoder and decoder works as expected

In [15]:
decode(encode(text[:100]))

'മീരാ..മീരാ..മീരാ മീരാ..\nകൃഷ്ണ കനയ്യാ,,കൃഷ്ണ കനയ്യാ..\nകൃഷ്ണ കനയ്യ കൃഷ്ണ ഹരേ..\nമീരാ...മീരാ....\nഗോവിന്ദ'

In [16]:
vocab_size = len(uniq_vocabs)
print( f"reduced vocabs to much smaller number : {vocab_size} from {encoder.n_vocab}")

reduced vocabs to much smaller number : 355 from 100277


In [18]:
data = torch.tensor(encode(text))

In [19]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [20]:


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [88]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(decode(xb[0].tolist()))
print('targets:')
print(yb.shape)
print(decode(yb[0].tolist()))

inputs:
torch.Size([4, 8])
�ഭഗാ�
targets:
torch.Size([4, 8])
ഭഗാന


In [89]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class MLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
m = MLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


In [94]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [101]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.7748582363128662


In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [22]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.248163 M parameters


In [31]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.2048, val loss 1.2070
step 300: train loss 1.2000, val loss 1.1982
step 600: train loss 1.1963, val loss 1.1959
step 900: train loss 1.1972, val loss 1.1948
step 1200: train loss 1.1880, val loss 1.1954
step 1500: train loss 1.1903, val loss 1.1823
step 1800: train loss 1.1832, val loss 1.1862
step 2100: train loss 1.1752, val loss 1.1836
step 2400: train loss 1.1790, val loss 1.1829
step 2700: train loss 1.1729, val loss 1.1768
step 3000: train loss 1.1721, val loss 1.1726
step 3300: train loss 1.1726, val loss 1.1704
step 3600: train loss 1.1744, val loss 1.1758
step 3900: train loss 1.1624, val loss 1.1737
step 4200: train loss 1.1718, val loss 1.1757
step 4500: train loss 1.1685, val loss 1.1670
step 4800: train loss 1.1647, val loss 1.1734
step 5100: train loss 1.1621, val loss 1.1619
step 5400: train loss 1.1594, val loss 1.1607
step 5700: train loss 1.1573, val loss 1.1640
step 6000: train loss 1.1501, val loss 1.1511
step 6300: train loss 1.1532, val loss 1

In [32]:
context = torch.full((1, 1), 87, dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


വാപന മാറിലെ
കനകക്കുന്നുപോരാടെ മദനദളചനയായി
നെറു പെറിയാമെന്നും ചുടക്കടയാലേഖ
വിടരൂ തുള്ളാൻ പെണ്ണീരിന്റെ സാഗരതീ ബിരേതം
നക്ഷത്രിപ്ദാക്കും ച൮യുന്നു നാട്ടുന്ന നഹൃദയമലരണിതളയടില്‍ മൂനയിനേഴോ
ജനന്മങ്ങള്‍ വാഴണ്ണി നിറപറകുകള്‍
നിരുകിവട കഴിയാളിന്നുണ്ടേ
മോഹംമെയ്യിൽ മൊയ്യാങ്കുല പൂങ്കൂടരാമോ
മുന്മിയതു നിലാവികാരങ്ങളാകെ, മതിമാതനാണിനി കാതില്‍
മദനസിയുന്നരിയില്‍ പോവാവില്‍
അരിയതോം അനുരാഗ്ളി അഴര്‍പൂങ്കാറ്റേ....

പ്രാരേ എടാന്‍
കുണിചിരടി മനസ്സിക്കീ ഗാനം
ഒരേ ഇതിലേ തിത്തിക നനയേ
മ..മഹി�്രീ...
അകമെത്തമേ പൂത്തു വിടരുന്നു
ആ )
കതതകളിലെ പഴകൾ യൂടലെല്ലേ...
നാം വീഴും കാത്തിയ കൊണ്ടു
നാണമാണോട്ടേ
മടയടിയേ പരവും  കലമേ കറങ്ങൂ...
കുനനേനോ നീ
നമ്മിതാ ചാരത്തെ
കടലേ താളങ്ങൾ അഴകിലെഴും....
ഇണമേണക്കുപാടി കരളിനെ കൊറ്റപ്പിഞ്ചസ്സാരേ
എന്നു മഴ പടിയല്ലൊരു മാടം മുങ്ങേല...
നിലവാക്കിളിയായൊതുപ്പോയ് കുളിതിരകളിലല്‍ താനേ
കാട്ടിനും കാറ്റേ ‍
ഹം...നീയ നൽക്കുമോ ...
വിരഭത്തി മയക്കാരേ...
കാറ്റേ കാറ്റേ താംബിക്കത്തിരിക്കു നല്‍കങ്ങളില്‍ (2)
ആണിക്കാറ്റും നാം കൊഴിയും താരം
പ്രണവും ദിവടടിയാലെന്തിച്ചവന്‍
ഗായൂജീരന്‍ രാഗജാനം...)
ഓലമായിചാര്‍ത്തിടാം എന്നും നീയും
നീലിത

# New section

In [115]:
encode('\n')

[87]